In [ ]:
# This script uses the ResNet-50 model provided by The MathWorks, Inc.
# Copyright (c) 2019, The MathWorks, Inc.
# Licensed under the terms described in LICENSE-BSD-3-Clause.txt

# Labelled Faces in the Wild (LFW) Dataset

In [25]:
from PIL import Image
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [30]:
# Paths to your real and fake image directories
real_images_path = r"C:\Users\nefel\Downloads\lfw-deepfunneled\lfw-deepfunneled"  
fake_images_path = r"C:\Users\nefel\Downloads\1m_faces_00\1m_faces_00"

# Collect all image paths in both real and fake directories
real_image_paths = glob.glob(os.path.join(real_images_path, '**', '*.jpg'), recursive=True) + \
              glob.glob(os.path.join(real_images_path, '**', '*.png'), recursive=True)
fake_image_paths = glob.glob(os.path.join(fake_images_path, "*.jpg"))  # or use *.png

# Verify the number of images in each dataset
print(f"Real images found: {len(real_image_paths)}")
print(f"Fake images found: {len(fake_image_paths)}")



Real images found: 13233
Fake images found: 10000


In [31]:
# Labels for real and fake images
real_labels = [0] * len(real_image_paths)  # 0 for real images
fake_labels = [1] * len(fake_image_paths)  # 1 for fake images

# Combine image paths and labels
all_image_paths = real_image_paths + fake_image_paths
all_labels = real_labels + fake_labels

print(f"Total images: {len(all_image_paths)}")

Total images: 23233


In [32]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load the image from the given path
        img_path = self.image_paths[idx]
        img = Image.open(img_path)
        
        # Apply transformations if specified
        if self.transform:
            img = self.transform(img)
        
        label = self.labels[idx]
        return img, label

In [48]:
from torchvision.transforms import InterpolationMode

transform = transforms.Compose([
    transforms.Resize(232, interpolation=InterpolationMode.BILINEAR),  # Resize the image to size [232]
    transforms.CenterCrop(224),  # Crop the image to size [224]
    transforms.ToTensor(),  # Convert the image to tensor and scale values to [0.0, 1.0]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize using resnet50 mean and std parameters
])

# Create the combined dataset
dataset = CustomImageDataset(image_paths=all_image_paths, labels=all_labels, transform=transform)

In [56]:

test_dataset = dataset

print(f"Test dataset size: {len(test_dataset)}")

Test dataset size: 23233


In [57]:
from torch.utils.data import DataLoader

# Create DataLoaders for each dataset split
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [51]:
from torchvision import models
import torch.nn as nn
import torch

model = models.resnet50(pretrained=False)  # Use `pretrained=False` as we will load custom weights
model.fc = nn.Linear(model.fc.in_features, 2)

In [52]:
checkpoint = torch.load(r"C:\Users\nefel\Downloads\results_CE_SGD_001_mom.pth", map_location=torch.device('cpu'))

C:\Users\nefel\AppData\Local\Temp\ipykernel_21372\901235479.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(r"C:\Users\nefel\Downloads\results_CE

In [53]:
# Load the weights into the model
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [58]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import torch.nn.functional as F

# Initialize lists to store predictions and true labels
all_preds = []
all_labels = []
all_probs = []

# Evaluate the model
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass through the model
        outputs = model(inputs)

        # Get the predicted class (0 or 1)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # Get probabilities for ROC AUC
        softmax_outputs = F.softmax(outputs, dim=1)
        all_probs.extend(softmax_outputs.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
roc_auc = roc_auc_score(all_labels, [prob[1] for prob in all_probs])  # Use the positive class probabilities

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

IndexError: list index out of range